# VNA V1

November 19th. Goal: Implement an import version of the signal source code, be able to render a "mockup" output streaming data

In [ ]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))
import ipywidgets as widgets
from IPython.display import display
import numpy as np 
from ipywidgets import Output

# Import functions from scripts
import sig_source
importlib.reload(sig_source)
from sig_source import SigSource

import numpy as np
import time
import threading
from bqplot import pyplot as plt
from threading import Thread

In [ ]:
rfsoc_button = widgets.Button(description="Update RFSOC Code")

out = Output()
def run_rfsoc(func):
    with out:
        out.clear_output
        try: 
            '''
            Code to run RFSOC by writing bitfile
            Placed inside this function to call only when on board
            '''
            from pynq import PL
            PL.reset() #important fixes caching issues which have popped up.
            import xrfdc #poorly documented library that handles interfacing to the RF data converter
            from pynq import Overlay  #import the overlay module
            ol = Overlay('./design_1_wrapper.bit')  #locate/point to the bit file
            import pprint
            pprint.pprint(ol.ip_dict)
            dma = ol.real_dma #might need to change name depending on what you called it
            rf = ol.usp_rf_data_converter_0 #might need to change name depending on what you called it
        except Exception as e:
            print(f"Error: {e}")

rfsoc_button.on_click(run_rfsoc)
display(widgets.VBox([widgets.Label(value="Update RFSOC Code"), rfsoc_button, out]))


In [ ]:
source = SigSource(start = 10000000, stop = 20000000000, resolution = 100)

start_stop_slider = widgets.FloatRangeSlider(
    value=[source.lowest_freq/(10**6), source.highest_freq/(10^6)],
    min= source.lowest_freq/(10**6),
    max= source.highest_freq/(10**6),
    step= 1,
    #description='Start Frequency (MHz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='1.0f',
    layout=widgets.Layout(width='500px') #Change layout width here!! 
)
center_slider = widgets.FloatSlider(value=source.center_freq/(10**6), min=source.lowest_freq/(10**6), max=source.highest_freq/(10**6), step=1)
span_slider = widgets.FloatSlider(value=((source.highest_freq - source.lowest_freq)/2)/10**6, min=1, max=((source.highest_freq - source.lowest_freq))/10**6, step=1)
resolution_slider = widgets.FloatSlider(value = 100, min = 0, max = 1000, step=1)
calc_selection = widgets.ToggleButtons(
    options=['Start-Stop', 'Center-Span'],
    description='Caculate Based On:',
    disabled=False,
    button_style=''
)
update_button = widgets.Button(description="Update Wave")

out = Output()
def update_function(change):
    with out:
        out.clear_output()
        try:
            if (calc_selection.value == "Start-Stop"):
                source.update_parameters(start = start_stop_slider.value[0]*10**6, 
                                        stop = start_stop_slider.value[1]*10**6, 
                                        resolution = resolution_slider.value)
            elif (calc_selection.value == "Center-Span"):
                source.update_parameters(center = center_slider.value*10**6,
                                        span = span_slider.value*10**6, 
                                        resolution = resolution_slider.value)
            
            start_stop_slider.value = (source.start/10**6, source.stop/10**6)
            center_slider.value = source.center/10**6
            span_slider.value = source.span/10**6
            #print("Start frequency: {} Stop Frequency: {}".format(start_stop_slider.value[0], start_stop_slider.value[1]) )
        except Exception as e:
            print(f"Error: {e}")


b1 = widgets.VBox([widgets.VBox([widgets.Label(value="Start Stop Slider (MHz)"), start_stop_slider]), 
                   widgets.HBox([widgets.VBox([widgets.Label(value="Center Slider (Mhz)"),center_slider]), 
                                 widgets.VBox([widgets.Label("Span Slider (Hz)"), span_slider])]),
                    widgets.VBox([widgets.Label(value="Points per Sweep"), resolution_slider]),
                    calc_selection,
                    update_button,
                    out],
                    layout=widgets.Layout(padding='20px 0'))

update_button.on_click(update_function)
display(b1)

In [ ]:
print(source.generate_freq_points())

In [ ]:
# # Fixed list of 100 points
# data_points = source.generate_freq_points()
# #y_data = freq_point_test
# #data_points = [i % 20 for i in range(100)]  # Example fixed data (modulus for variation)

# # Generate corresponding time markers
# time_markers = list(range(len(data_points)))  # X-axis represents time markers (0, 1, ..., 99)

# # Initialize the plot
# fig = plt.figure(title="Current Frequency Output Plot", animation_duration=200)
# line = plt.plot([], [], colors=["blue"])  # Initialize empty plot
# plt.ylim(min(data_points) - 1, max(data_points) + 1)  # Y-axis limits based on data
# plt.xlim(0, 10)  # Initial X-axis visible range

# # Toggle button to start/stop the plot
# is_running = widgets.ToggleButton(
#     value=True,
#     description="Running",
#     icon="play",
#     tooltip="Start/Stop the live plot",
# )

# # Display the toggle button and plot
# display(is_running, fig)

# # Function to update the plot
# def update_plot():
#     window_size = 100  # Number of points visible on the plot at a time
#     i = 0  # Start index
#     while True:
#         if is_running.value:
#             # Get the current window of data
#             start_idx = max(0, i - window_size)
#             visible_x = time_markers[start_idx:i + 1]
#             visible_y = data_points[start_idx:i + 1]
            
#             # Update the plot data
#             line.x = visible_x
#             line.y = visible_y
            
#             # Scroll the X-axis window
#             plt.xlim(visible_x[0], visible_x[-1])
            
#             i += 1
#             if i >= len(data_points):
#                 i = 0  #restart datapoints
            
#             time.sleep(0.1)  # Update every 100ms
#         else:
#             time.sleep(0.1)  # Pause updates when not running

# # Run the update function in a separate thread
# thread = threading.Thread(target=update_plot, daemon=True)
# thread.start()


In [ ]:

# Simulate incoming sensor data (for now, a sine wave with noise)
def generate_fake_sensor_data(t):
    # Simulate a noisy sine wave (sensor data)
    noise = np.random.normal(0, 0.2)  # Gaussian noise with mean 0 and std 0.2
    return np.sin(t) + noise

# Initialize data lists
time_data = []  # Time data (X-axis)
sensor_data = []  # Sensor data (Y-axis)

# Initialize the plot
fig = plt.figure(title="Real-time Sensor Data", animation_duration=200)
line = plt.plot([], [], colors=["blue"])  # Initial empty plot

plt.ylim(-2, 2)  # Set Y-axis range to capture sine wave + noise
plt.xlim(0, 10)  # Initial X-axis range, will update dynamically

# Function to update the plot with new sensor data
def update_plot():
    time_step = 0.1  # Update interval (time step for sine wave)
    t = 0  # Initial time
    while True:
        if is_running.value:
            # Generate fake sensor data
            sensor_value = generate_fake_sensor_data(t)
            
            # Append the data
            time_data.append(t)
            sensor_data.append(sensor_value)
            
            # Update the plot with new data
            line.x = time_data
            line.y = sensor_data
            
            # Scroll the X-axis window (show the last 50 time points)
            plt.xlim(max(0, t - 5), t + 5)
            
            # Move the time forward
            t += time_step
            
            time.sleep(0.1)  # Simulate data coming in every 100ms

# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=True,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')